In [1]:
import os,sys
sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..')))
import warnings
warnings.filterwarnings("ignore")

In [2]:
import scanpy as sc
import pandas as pd
import CPS

In [3]:
opt = CPS.config()
args = opt.parse_args(['--seed', '2026'])
args.prep_scale = False
args.sigma=0.8
args.dataset_path = '/mnt/d/Dataset/SRT_Dataset/1-DLPFC/'
args

Namespace(batch_size=256, clusters=7, coord_dim=2, dataset_path='/mnt/d/Dataset/SRT_Dataset/1-DLPFC/', decoder_latent=[256, 512, 1024], distill=1.0, dropout=0.2, er_w=0.05, flow='source_to_target', freq=32, gpu=0, hvgs=3000, inr_latent=[256, 256, 256], k_list=[0, 1, 2, 3, 4, 5, 6, 7], latent_dim=64, lr=0.001, max_epoch=1000, max_neighbors=6, n_spot=0, num_heads=4, prep_scale=False, radius=150, seed=2026, self_loops=True, sigma=0.8, weight_decay=0.0001)

In [4]:
sections = ['151507', '151508', '151509', '151510',
            '151669', '151670', '151671', '151672',
            '151673', '151674', '151675', '151676']
mask_ratio = 0.5

In [5]:
for section in sections:
    CPS.set_random_seed(args.seed)
    adata = sc.read_visium(os.path.join(args.dataset_path+section))
    Ann_df = pd.read_csv(os.path.join(args.dataset_path+'1-DLPFC_annotations/', section+'_truth.txt'), 
                        sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth']
    adata.var_names_make_unique()
    adata.layers["counts"] = adata.X.copy()
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=args.hvgs)
    adata_hvg = adata[:, adata.var['highly_variable']].copy()
    sc.pp.normalize_total(adata_hvg, target_sum=1e4)
    sc.pp.log1p(adata_hvg)
    sc.pp.scale(adata_hvg)
    
    counts_layer = adata_hvg.layers['counts']
    adata_hvg.obsm['hvg_features'] = adata_hvg.X
    adata_hvg.obsm['hvg_counts'] = counts_layer.toarray()
    
    spatial_data = CPS.SpatialGraphBuilder(args)
    train_data, test_data, train_indices, test_indices = spatial_data.spots_perturb(adata_hvg, mask_ratio=mask_ratio)
    
    cps_model = CPS.CPSTrainer(args)
    cps_model.fit(train_data, verbose=False)
    
    metrics = cps_model.evaluate_spots_imputation(test_data=test_data, 
                                                  experiment_name=f'DLPFC_SI_{section}',
                                                  use_log1p=True)
    

  0%|          | 0/1000 [00:00<?, ?it/s]

Performing spots imputation, number of test spots: 2113

Computing metrics on **Log1p Transformed** data...

Spatial imputation metrics [Log1p Transformed]

[global flatten metrics]
MSE:           0.040319
RMSE:          0.200795
MAE:           0.095889
R²:            0.629494
Pearson:       0.796263
Spearman:      0.410124

[sample mean metrics]
CS:               0.791264
CS nonzero:       0.883883
Pearson:          0.774865
Pearson nonzero:  0.839193
Spearman:         0.396337
Spearman nonzero: 0.598269
SAM:              37.494373°
SAM nonzero:      27.885672°
MSE:              0.040319
MSE nonzero:      0.287384
MAE:              0.095889
MAE nonzero:      0.469732

 num of sample: 2113.0
Metrics saved to: ./results/DLPFC_SI_151507_metrics.json


  0%|          | 0/1000 [00:00<?, ?it/s]

Performing spots imputation, number of test spots: 2192

Computing metrics on **Log1p Transformed** data...

Spatial imputation metrics [Log1p Transformed]

[global flatten metrics]
MSE:           0.036804
RMSE:          0.191843
MAE:           0.086452
R²:            0.624886
Pearson:       0.793813
Spearman:      0.380277

[sample mean metrics]
CS:               0.792166
CS nonzero:       0.874859
Pearson:          0.777713
Pearson nonzero:  0.862321
Spearman:         0.365058
Spearman nonzero: 0.589702
SAM:              37.481747°
SAM nonzero:      28.971760°
MSE:              0.036804
MSE nonzero:      0.304010
MAE:              0.086452
MAE nonzero:      0.487889

 num of sample: 2192.0
Metrics saved to: ./results/DLPFC_SI_151508_metrics.json


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Performing spots imputation, number of test spots: 1836

Computing metrics on **Log1p Transformed** data...

Spatial imputation metrics [Log1p Transformed]

[global flatten metrics]
MSE:           0.070011
RMSE:          0.264595
MAE:           0.149248
R²:            0.665458
Pearson:       0.820164
Spearman:      0.536975

[sample mean metrics]
CS:               0.821968
CS nonzero:       0.907334
Pearson:          0.793267
Pearson nonzero:  0.798321
Spearman:         0.523540
Spearman nonzero: 0.655926
SAM:              34.373386°
SAM nonzero:      24.860518°
MSE:              0.070011
MSE nonzero:      0.256715
MAE:              0.149248
MAE nonzero:      0.432442

 num of sample: 1836.0
Metrics saved to: ./results/DLPFC_SI_151674_metrics.json


  0%|          | 0/1000 [00:00<?, ?it/s]

Performing spots imputation, number of test spots: 1796

Computing metrics on **Log1p Transformed** data...

Spatial imputation metrics [Log1p Transformed]

[global flatten metrics]
MSE:           0.052575
RMSE:          0.229291
MAE:           0.118101
R²:            0.588186
Pearson:       0.773991
Spearman:      0.453085

[sample mean metrics]
CS:               0.758569
CS nonzero:       0.876852
Pearson:          0.730395
Pearson nonzero:  0.741669
Spearman:         0.435784
Spearman nonzero: 0.598196
SAM:              40.240089°
SAM nonzero:      28.735066°
MSE:              0.052575
MSE nonzero:      0.277393
MAE:              0.118101
MAE nonzero:      0.458571

 num of sample: 1796.0
Metrics saved to: ./results/DLPFC_SI_151675_metrics.json


  0%|          | 0/1000 [00:00<?, ?it/s]

Performing spots imputation, number of test spots: 1730

Computing metrics on **Log1p Transformed** data...

Spatial imputation metrics [Log1p Transformed]

[global flatten metrics]
MSE:           0.057021
RMSE:          0.238790
MAE:           0.125022
R²:            0.606066
Pearson:       0.785089
Spearman:      0.473182

[sample mean metrics]
CS:               0.779062
CS nonzero:       0.882963
Pearson:          0.750600
Pearson nonzero:  0.751647
Spearman:         0.459635
Spearman nonzero: 0.616905
SAM:              38.465202°
SAM nonzero:      27.998103°
MSE:              0.057021
MSE nonzero:      0.276033
MAE:              0.125022
MAE nonzero:      0.455126

 num of sample: 1730.0
Metrics saved to: ./results/DLPFC_SI_151676_metrics.json
